In [1]:
from PIL import Image
from keras.applications.vgg16 import preprocess_input
import base64
from io import BytesIO
import json
import random
import cv2
from keras.models import load_model
import numpy as np

In [2]:
from keras.preprocessing import image
model = load_model(r'models/vgg16_model.h5')

In [7]:
import mediapipe as mp

mpFaceDetection = mp.solutions.face_detection
faceDetection = mpFaceDetection.FaceDetection(0.45)

def face_extractor(frame):
    
    imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = faceDetection.process(imgRGB)
    
    
    if(results.detections):
        for id, detection in enumerate(results.detections):
            
            bboxC = detection.location_data.relative_bounding_box
            img_h, img_w, img_c = frame.shape
            
            xmin = int(bboxC.xmin * img_w)
            ymin = int(bboxC.ymin * img_h)
            w = int(bboxC.width * img_w)
            h = int(bboxC.height * img_h)
            
            
            bbox = (int(bboxC.xmin * img_w), int(bboxC.ymin * img_h), int(bboxC.width * img_w), int(bboxC.height * img_h))

            new_frame = frame[ymin:ymin+h, xmin:xmin+w]
            
    
            return new_frame, bbox
    else:
        
        return np.array([]), []
        

In [10]:
video_capture = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX

while True:
    _, frame = video_capture.read()
    
    face, bbox=face_extractor(frame)
    
    if(face.size):
        face = cv2.resize(face, (224, 224)) #Resizing into 128x128 because we trained the model with this image size.
        im = Image.fromarray(face, 'RGB')

        img_array = np.array(im)

        #Our keras model used a 4D tensor, (images x height x width x channel)
        #So changing dimension 128x128x3 into 1x128x128x3 
        img_array = np.expand_dims(img_array, axis=0)


        pred = model.predict(img_array)
        prediction = np.argmax(pred)
        #print(prediction)


    

        cv2.rectangle(frame, bbox, (57, 255, 20), 2)
        cv2.putText(frame, str(prediction), (bbox[0],bbox[1]-20), font, 0.8, (57, 255, 20), 2)
        
    else:
        cv2.putText(frame, 'Face not found', (10, 50), font, 0.8, (57, 255, 20), 2)

    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

        
video_capture.release()
cv2.destroyAllWindows()
